In [2]:
import os
import psycopg2 as psy
import pandas as pd
import numpy as np
import time

from py_package import py_module as pm
from datetime import datetime as dt
from datetime import timedelta

from random   import choice

In [4]:
#''' Check if file exists on path. Move existing file to ARCH. Create new input for calendar_fact.'''

# a = py_package.py_module.atest(5)
# print(a)

connection_dict = { "user" : "szymonbocian", \
                    "password" : "", \
                    "host" : "localhost", \
                    "port" : "5432", \
                    "database" : "dwh_call_center"
                  }
# print(connection_dict)

a = pm.DbConnection("")


{'user': 'szymonbocian', 'password': '', 'host': 'localhost', 'port': '5432', 'database': 'dwh_call_center'}


In [106]:
# Setup script input parameters

week_of_year = 10
value_year = 2019
employee_number = 100
work_start_time = 6 # 06:00 am
work_end_time = 22  # 10:00 pm

def get_time(time):
    return ('0' + str(time) + ':00')[-5:]

# work_start_time = get_time(work_start_time)
# work_end_time = get_time(work_end_time)

print(work_start_time)
print(work_end_time)

6
22


In [4]:
main_path    = "Documents/CallCenterStaffing"
path_to_arch = "Arch/calendar"
path_to_file = "Input"
file_name    = "consultant_work_calendar_fact.csv"

now          = dt.now()

path_to_file = os.path.join(os.path.expanduser("~"),main_path,path_to_file,file_name)
path_to_arch = os.path.join(os.path.expanduser("~"),main_path,path_to_arch)

def is_path_not_correct(path):
    return not(os.path.exists(path))

def is_path_correct(path):
    return os.path.exists(path)

if is_path_not_correct(path_to_file):
    raise Exception("Input file doesn't exist")

if is_path_not_correct(path_to_arch):
    raise Exception("Archive directory doesn't exist")

time_stamp = "_" + str(now)[:10]
arch_file_name = 'arch_' + file_name.replace(".csv",time_stamp + ".csv")
 
os.rename(path_to_file, os.path.join(path_to_arch, arch_file_name))

In [58]:
# connection to database and return list of column column_list
try:
    connection = psy.connect( user = "szymonbocian",
                              password = "",
                              host = "localhost",
                              port = "5432",
                              database = "dwh_call_center")

    cursor = connection.cursor()

    cursor.execute("""
        SELECT column_name 
        FROM INFORMATION_SCHEMA.columns 
        WHERE table_schema = 'stg' AND table_name = 'consultant_work_calendar_fact';
    """)
    column_list = cursor.fetchall()
    
    cursor.execute("""
        SELECT consultant_business_key 
        FROM pro.consultant_dim
        WHERE active = True;
    """)
    consultant_business_key_list = cursor.fetchall()
    
    cursor.execute("""
        SELECT office_business_key
        FROM pro.office_dim 
        WHERE active = True;
    """)
    office_business_key_list = cursor.fetchall()

    cursor.execute( "SELECT " + \
                    "MAX(start_of_week_mon_date), MAX(end_of_week_mon_date) " + \
                    "FROM tech.date_dim " + \
                    "WHERE iso_week_of_year = " + str(week_of_year) + " AND year_value = " + str(value_year) + ";"
                  )
    week_list = cursor.fetchall()
        
except (Exception, psy.Error) as error :
    print ("Error while connecting to PostgreSQL", error)
finally:
        if(connection):
            cursor.close()
            connection.close()

column_list = [''.join(c for c in s if c not in '(),') for s in column_list]
consultant_business_key_list = [''.join(c for c in s if c not in '(),') for s in consultant_business_key_list]
office_business_key_list = [''.join(c for c in s if c not in '(),') for s in office_business_key_list]
print(column_list)
print(consultant_business_key_list)
print(office_business_key_list)
print(week_list)

['work_calendar_business_key', 'consultant_business_key', 'office_business_key', 'work_date', 'work_start_time', 'work_end_time']
['AA032048000E', 'WP056056004E', 'MM040048004E', 'PM064064004E', 'EG040040004E', 'MG072024004E', 'LD056088004E', 'KK048064004E', 'ET072096004E', 'JP056040004E', 'SK064048004E', 'BG048080004E', 'KP048064004E', 'MA040088004E', 'OK056088004E', 'KK056088004E', 'DP040056004E', 'RK032064004E', 'BF040048004E', 'SK048040004E', 'MH072040004E', 'NK056072004E', 'WC056056004E', 'ZK064072004E', 'HK048064004E', 'IK024048004E', 'BG072048004E', 'OK048024004E', 'IK032072004E', 'KG072048004E', 'JK056056004E', 'AZ040032004E', 'NL048056004E', 'RR064040004E', 'BJ040056004E', 'LK032048004E', 'MK064072004E', 'CO056040004E', 'KM064072004E', 'FN048056004E', 'EI032080004E', 'MF056032004E', 'MT048088004E', 'PK048072004E', 'KB056064004E', 'SP072048004E', 'AM040056004E', 'MK040040004E', 'FB064040004E', 'JA024088004E', 'MU056064004E', 'MG040040004E', 'JG064040004E', 'TF040072004E', 'MN08

In [109]:
# df_cons = pd.DataFrame({
#     "consultant_business_key" : consultant_business_key_list,
#     "office_business_key" : np.random.choice(office_business_key_list, len(consultant_business_key_list)),
# })

start_time = time.mktime(datetime.timestamp(week_list[0][0]) + timedelta(hours = work_start_time)
print(start_time)
df_date = pd.DataFrame({
    "work_date" : pd.date_range(start = week_list[0][0], end = week_list[0][1])
})

# work_time = work_end_time - work_start_time



# df_time = pd.DataFrame({
#     "work_time" : pd.date_range('2019-01-10', periods = work_time, freq = '8')
# })
# df_time
# print(week_list[0][0])
# print(week_list[0][1])

TypeError: descriptor 'timestamp' requires a 'datetime.datetime' object but received a 'datetime.date'

In [51]:
# liczba_dni * liczba_godzin * liczba_pracowników * średnia_liczba_połączeń_na_godzinę
# 7 dni w tygodniu , od 6:00 do 22:00 ( 16 godzin )
# 100 konsultantów
# 5 - 8 połączeń na godzinę / konsultant
# dane z 4 tygodni i prognozowanie na ich podstawie
#tydzień
# 7*16*100*5     # = 56 000
# 7*16*100*8     # = 89 600
# 4 tygodnie     
# 4*7*16*100*5    # = 224 000
# 4*7*16*100*8    # = 358 400


358400